In [30]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [31]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [32]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255, x_test / 255

# Add a channels dimention (?)
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


In [33]:
# Agrupar e embaralhar conjunto de dados

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [34]:
# Modelo usando a API de subclassificação do modelo Keras:

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


In [35]:
model = MyModel()

In [36]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [37]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy(name="test_accuracy")

In [38]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradiant = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradiant, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

    

In [39]:
def test_step(images, labels):
    predicitons = model(images, training=False)
    t_loss = loss_object(labels, predicitons)

    test_loss(t_loss)
    test_accuracy(labels, predicitons)


In [40]:
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f"Epoch {epoch + 1} "
        f"Loss {train_loss.result()} "
        f"Accuracy {train_accuracy.result() * 100} "
        f"Test Loss {test_loss.result()} "
        f"Test Accuracy {test_accuracy.result() * 100}"

    )


Epoch 1 Loss 0.1316116750240326 Accuracy 133.96151733398438 Test Loss 0.06468438357114792 Test Accuracy 129.3116912841797
Epoch 2 Loss 0.041758179664611816 Accuracy 121.68923950195312 Test Loss 0.05175282061100006 Test Accuracy 117.94628143310547
Epoch 3 Loss 0.02185252122581005 Accuracy 109.26786804199219 Test Loss 0.054257091134786606 Test Accuracy 102.89379119873047
Epoch 4 Loss 0.012361425906419754 Accuracy 97.92263793945312 Test Loss 0.05986785888671875 Test Accuracy 95.97245788574219
Epoch 5 Loss 0.009958108887076378 Accuracy 90.02556610107422 Test Loss 0.07235702872276306 Test Accuracy 83.80916595458984
